# Analytics report on expansion of products

## Breakdown of the business problem
Our company decides to start selling products internationally. Therefore, there is a need in the analysis of the market around the world to be able to understand which countries will be the best choice in sense of production and profit.
For this problem several aspects were analyzed.
1. The demand on the sport products around the world.
2. The population and its growth rate around the world.
3. The cost of living in the chosen countries.
4. The comparison of GNI and GDP in the chosen countries to understand in which countries international business is more frequent.

The analysis showed several useful insights that helped to derive the following insights and recommendations to the countries where out company should expand:

**1. The company should expand to such countries as: Nigeria, Bangladesh.**
This conclusion is derived from their significant share of world's population, significant growth rate of population and high trust international investors, which is based on high GNI/GDP rate. Moreover, Nigeria is a preferable option to Bangladesh as all of the analyzed features are better there.

**2. China can be also included in this list.**
The drawbacks it has are not high enough growth rate of population and high cost of living. Nevertheless, China has the higher population share in the world and high international investors trust in China's economy. Moreover, China is one of the fastest growing economies with high GDP, which means that people there can cover the cost of living and will be able to spend money on additional sports products. 

**3. None of these countries are in the top countries for sales.**
Nevertheless, it means that it is a new market for sports clothing, which gives lots of opportunities as it means that there is less competition in the market from other sports companies.**

Assumptions used in this report:
1. Our company is located in Europe. Therefore, all of the countries included in Europe (European Unity) will be excluded from the analysis as our products are already presented in the region.
2. Year 2020 is taken as a base for this analysis as there is no data after that year. Analysis is done taking into account the world's situation in 2020. 
3. For the cost of living index by country 2020 by doing the online research on the data and confirming the quality of the data.
4. Population_by_country_2020 is used as the source for population data as it is more close to the data that can be found online.
5. Less developing countries are excluded for this analysis.
6. It is not the full analysis as there is no data on income of people. Therefore, it is hard to compare the real wealth of people, based only on their cost of living.
7. Some of the NA values are neglected in the data as they do not influence the analysis.

More detailed report represented below.

In [ ]:
#Importing all of the needed packages
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import seaborn as sns
import plotly.express as px
import chardet

!pip install mtranslate
from mtranslate import translate

import warnings
warnings.filterwarnings('ignore')

### Analysis of the sales

It should be mentioned that sales data is represented for the years 2015-2018, while other data is based on the year 2020. This fact will be taken into account for the final conclusions.

In [ ]:
#Uploading the data
#Coding of the file returns the mistake. Therefore, different method from usual read_csv will be used.

with open('./DataCoSupplyChainDataset.csv/DataCoSupplyChainDataset.csv', 'rb') as f:
    enc = chardet.detect(f.read())
    
d0 = pd.read_csv('./DataCoSupplyChainDataset.csv/DataCoSupplyChainDataset.csv', encoding = enc['encoding'])

#Choosing only interesting columns
data_sales = d0[['Days for shipping (real)', 'Benefit per order', 'Category Name', 'Customer Country', 'Customer Segment', 'Department Name', 'Market', 'Order Country', 'Order Item Profit Ratio', 'Order Item Quantity', 'Sales','Order Item Total','Order Profit Per Order','Order Region']]
#data_sales.head()

In [ ]:
#Checking the data
data_sales.info()

In [ ]:
#Creating a function to check for missing values in all of the columns in the data
def mis_val(data):
    for l in data.columns:
        n_miss = data[[l]].isna().sum().sum()
        perc = round(n_miss/len(data)*100,2)
        print(f'Missing values for \033[1m{l}\033[0m is {perc}% ')

In [ ]:
mis_val(data_sales)

In [ ]:
#Choosing only data related to sport or sport clothing
data_sales = data_sales[data_sales['Department Name'].str.contains('Apparel|Fitness|Outdoors|Golf')]
data_sales = data_sales[data_sales['Category Name'].str.contains('Clothing|Baseball & Softball|Basketball|Hockey|Lacrosse|Soccer|Sporting Goods|Tennis & Racquet|Apparel|Gloves|Golf')]
data_sales.head()

In [ ]:
#Creating pivot table for choosing top contries by sales and benefit per order and looking at their continent and regions
data_sales_grouped = data_sales.pivot_table(['Sales','Benefit per order'], index = ['Order Region','Order Country']).nlargest(20, 'Benefit per order').sort_values(by =['Sales','Benefit per order','Order Region','Order Country'],ascending=[False, False, True, True])
data_sales_grouped

In [ ]:
#Creating a list with names of the countris for future analysis
list_cont = []
for m in data_sales_grouped.index.get_level_values(1):
    list_cont.append(m)
list_cont

col_trans = pd.DataFrame(list_cont, columns= ['Countries'])

In [ ]:
#As the names of the countries is written in Spainish, changing them to English using additional package.
def translate_text(text):
    return translate(text, 'es:en')

col_trans['Countries'] = col_trans['Countries'].apply(lambda x: translate_text(x))
col_trans['Countries'] = col_trans['Countries'].str.capitalize()

In [ ]:
#We keep these countries in mind and go for further analysis of population and its growth rates.

### Analysis of population

#### Regions

In [ ]:
#Firstly, uploading the data for regions for future merge with other columns
data_reg = pd.read_csv('./World Bank/Countries_and_continents_of_the_world.csv')

#Keeping only interesting columns 
data_reg = data_reg[['Country Name', 'Climate_short_desc', 'Phones (per 1000)','Continent', 'Birthrate','Country Code', 'latitude','longitude','Region']]
data_reg.head()

In [ ]:
#Creating the function for cutting the names for future merging tables as the name of the countries are filled in differently
def name_cut(data, col_name):
    data['Country_Name_cut'] = data[col_name].astype(str).str.replace(' ','').str.lower()
    data['Country_Name_cut'] = data['Country_Name_cut'].str.replace('& ','and')
    return data

name_cut(data_reg, 'Country Name').head()

In [ ]:
#Checking for missing values
mis_val(data_reg)

In [ ]:
#Checking types of the data
data_reg.info()

In [ ]:
#Changing types for needed one and for future cleaning
data_reg['Phones (per 1000)'] = data_reg['Phones (per 1000)'].str.replace(',','.').astype(float).round(2)
data_reg['Birthrate'] = data_reg['Birthrate'].str.replace(',','.').astype(float).round(2)

In [ ]:
#Creating the function to fill null values for numerical values with k-Neighbours
def kNNneig(col_name, data, n):
    data_list = [col_name]
    newvalues = data[data_list]
    impKNN = KNNImputer(n_neighbors=n)
    newval = impKNN.fit_transform(newvalues)
    df_new = pd.DataFrame(newval, columns=data_list, index = newvalues.index)
    return df_new


In [ ]:
data_reg_ph = kNNneig('Phones (per 1000)', data_reg, 10)
data_reg_b = kNNneig('Birthrate', data_reg, 10)

In [ ]:
#Merging new data with initial table, leave only needed columns and renaming the columns after merging
data_reg = pd.merge(data_reg, data_reg_ph, left_index=True, right_index=True)
data_reg = pd.merge(data_reg, data_reg_b, left_index=True, right_index=True)
data_reg = data_reg[['Country Name', 'Climate_short_desc', 'Phones (per 1000)_y', 'Continent', 'Birthrate_y', 'Country Code', 'latitude', 'longitude','Region','Country_Name_cut']]
data_reg.rename(columns = {'Climate_short_desc': 'Climate', 'Phones (per 1000)_y': 'Phones, per 1000','Birthrate_y':'Birthrate'}, inplace = True)


In [ ]:
#Fill null data for column Climate as 'No data' as we cannot really apply anything to it without mistake. Dropping lines
#without country code. Rounding the values for several columns
data_reg = data_reg.fillna('No data')
data_reg.dropna(subset = ['Country Code'], inplace = True)
data_reg['Phones, per 1000'] = data_reg['Phones, per 1000'].round(2)
data_reg['Birthrate'] = data_reg['Birthrate'].round(2)
data_reg.head()

In [ ]:
#Checking for missing values and continue with analysis of population
mis_val(data_reg)

#### Popilation

In [ ]:
#Uploading the data
data_pop = pd.read_csv('./World Bank/population-by-country-2020/population_by_country_2020.csv')

#Cleaning the data
data_pop.rename(columns = {'Country (or dependency)': 'Country Name'}, inplace = True)
data_pop[['Yearly Change', 'Unit']] = data_pop['Yearly Change'].str.split(' ', expand = True)
data_pop['Yearly Change'] = data_pop['Yearly Change'].astype(float)
data_pop[['Urban Pop %', 'Unit']] = data_pop['Urban Pop %'].str.split(' ', expand = True)
data_pop.loc[data_pop['Urban Pop %'].str.contains('N.A.') == True, 'Urban Pop %'] = np.nan
data_pop.loc[data_pop['Fert. Rate'].str.contains('N.A.') == True, 'Fert. Rate'] = np.nan
data_pop.loc[data_pop['Med. Age'].str.contains('N.A.') == True, 'Med. Age'] = np.nan
data_pop[['World Share', 'Unit']] = data_pop['World Share'].str.split(' ', expand = True)
data_pop['Urban Pop %'] = data_pop['Urban Pop %'].astype(float)
data_pop.head()

In [ ]:
#Checking for the missing values
mis_val(data_pop)

In [ ]:
#As there is no info for three columns and population in these countries is small, made a decision to exclude them from 
#the analysis.
data_pop.dropna(subset = ['Migrants (net)', 'Fert. Rate', 'Med. Age'], how = 'all', inplace = True)

#Checking missing values again
mis_val(data_pop)

In [ ]:
#As we can see there is still null values in the column Urban Pop %. Using k-Neigbours function:

data_pop_knn = kNNneig('Urban Pop %', data_pop, 10)

In [ ]:
#Merging new data and exluding previous data from table. Renaming columns
data_pop = data_pop.merge(data_pop_knn, left_index=True, right_index=True)
data_pop = data_pop[['Country Name','Population (2020)','Yearly Change','Net Change','Density (P/Km²)','Migrants (net)','Fert. Rate','Med. Age','Urban Pop %_y','World Share']]
data_pop.rename(columns = {'Population (2020)':'Population', 'Urban Pop %_y': 'Urban Pop %', 'Yearly Change': 'Yearly Change %', 'World Share':'World Share %'}, inplace = True)

In [ ]:
#Creating the column for country names
name_cut(data_pop, 'Country Name')

#Creating the function to exclude EU from the data due to our assumption
def eu_flag(data, colm_name):
    data.loc[data[colm_name].str.contains('austria|belgium|bulgaria|croatia|republicofcyprus|czechrepublic|denmark|estonia|finland|france|germany|greece|hungary|ireland|italy|latvia|lithuania|luxembourg|malta|netherlands|poland|portugal|romania|slovakia|slovenia|spain|sweden') == True, 'EU'] = 1
    data.loc[data[colm_name].str.contains('austria|belgium|bulgaria|croatia|republicofcyprus|czechrepublic|denmark|estonia|finland|france|germany|greece|hungary|ireland|italy|latvia|lithuania|luxembourg|malta|netherlands|poland|portugal|romania|slovakia|slovenia|spain|sweden') == False, 'EU'] = 0
    data['EU'] = data['EU'].astype(int)
    return data
  
eu_flag(data_pop, 'Country_Name_cut')

#Excluding EU countries from data. Changing index
data_pop = data_pop[data_pop['EU'] == 0]
data_pop.index = np.arange(1, len(data_pop) + 1)

In [ ]:
#Merging population data with regions 
data_pop = pd.merge(data_pop, data_reg, how= 'left', on = 'Country_Name_cut')
data_pop.head()

In [ ]:
#Renaming columns
data_pop.rename(columns = {'Country Name_x':'Country Name'}, inplace = True)

In [ ]:
#Checking the types of data
data_pop.info()

In [ ]:
#Dropping the lines with null values in 2 columns 
data_pop.dropna(subset = ['Climate', 'Continent'], how = 'all', inplace = True)

#Checking for null values
mis_val(data_pop)

In [ ]:
#Changing values to int/float type for numbers
data_pop['Fert. Rate'] = data_pop['Fert. Rate'].astype(float)
data_pop['Med. Age'] = data_pop['Med. Age'].astype(float)
data_pop['World Share %'] = data_pop['World Share %'].astype(float)
data_pop['Population'] = data_pop['Population'].astype(int)
data_pop.info()

In [ ]:
#Leaving only needed columns
data_pop = data_pop[['Country Name', 'Population', 'Yearly Change %', 'Net Change', 'Density (P/Km²)', 'Migrants (net)', 'Fert. Rate', 'Med. Age', 'Urban Pop %', 'World Share %', 'EU', 'Climate', 'Phones, per 1000','Continent','Region','Birthrate', 'Country Code', 'latitude','longitude']]
#data_pop.head()

In [ ]:
#Looking at the structure of the data
round(data_pop.describe(),2)

In [ ]:
#Sorting the data to look at the top countries according to their population and growth rate
data_pop.sort_values(by=['World Share %', 'Urban Pop %', 'Yearly Change %'], ascending=[False, False, False]).head()

In [ ]:
# Excluding China and India as these countries are outliers. But still taking them into account for future analysis
data_pop2 = data_pop.copy()

In [ ]:
data_pop2 = data_pop2[data_pop2['Country Name']!= 'China']
data_pop2 = data_pop2[data_pop2['Country Name']!= 'India']
data_pop2.reset_index(drop = True, inplace = True)
data_pop2.head(8)

In [ ]:
#Plotting map using plotly
fig = px.scatter_geo(data_pop2, lat = 'latitude', lon = 'longitude', size = 'Population', hover_name = 'Country Name', color = 'Population')
fig.update_layout(title = 'Population', title_x = 0.5)
fig.show()

In [ ]:
#Checking for Kurtosis and Skewness of data without outliers
data_pop2.kurt()

In [ ]:
data_pop2.skew()

In [ ]:
#Checking the distribution for continents and regions
d1 = data_pop2.groupby('Continent')['Population'].sum()
d1.astype(int).sort_values(ascending=False)

In [ ]:
d2 = data_pop2.groupby('Region')['Population'].sum()
d2.astype(int).sort_values(ascending=False)

In [ ]:
#Creating pivot table to check for population for continents and regions together
data_pop2.pivot_table('Population', index = ['Continent','Region']).astype(int).round(2).sort_values(by =['Continent','Population','Region'],ascending=[True,False, True])

In [ ]:
#Choosing top contries by population and looking at their continent and regions
dt_pop_pt = data_pop2.pivot_table('Population', index = ['Continent','Region','Country Name']).nlargest(15,'Population').sort_values(by =['Continent','Population','Region'],ascending=[True,False, True])
dt_pop_pt

In [ ]:
#Saving countries names from pivot table
list_cnt = []
for m in dt_pop_pt.index.get_level_values(2):
    list_cnt.append(m)
#list_cnt

In [ ]:
#Adding China and India to the list for future analysis
list_cnt2 = []
list_cnt2 = list_cnt.copy()
list_cnt2.append('China')
list_cnt2.append('India')
#list_cnt2

In [ ]:
#Create the data frame only with chosen countries
#Data with China and India
data_chos = data_pop[data_pop['Country Name'].isin(list_cnt2)]
#Data without China and India
data_chos2 = data_pop2[data_pop2['Country Name'].isin(list_cnt2)]

In [ ]:
data_chos.describe().round(2)

In [ ]:
#Creating the map with chosen countries for better visualization.
#Including China and India
fig = px.scatter_geo(data_chos, lat = 'latitude', lon = 'longitude', size = 'Population', hover_name = 'Country Name', color = 'Population')
fig.update_layout(title = 'Population', title_x = 0.5)
fig.show()


In [ ]:
#Excluding China and India
fig = px.scatter_geo(data_chos2, lat = 'latitude', lon = 'longitude', size = 'Population', hover_name = 'Country Name', color = 'Population')
fig.update_layout(title = 'Population', title_x = 0.5)
fig.show()

In [ ]:
data_chos2.describe().round(2)

In [ ]:
#Leaving only interesting columns and sorting them in descending order to see top countries
data_chos_f = data_chos[['Country Name', 'Yearly Change %', 'World Share %', 'Urban Pop %',]].copy()
data_chos_f.sort_values(by=['World Share %', 'Urban Pop %', 'Yearly Change %'], ascending=[False, False, False])

In [ ]:
#Creating plot for better visualization
axes = data_chos_f.plot.bar(x = 'Country Name',rot = 45, subplots = True)

#From this graph we can see that several countries has a high world population share and that sufficient part of it is
#urban. It means that it is easier to target marketing company, more people will have access to shops and
#more people will be interested in sport clothes to exercises in cities areas, as they most likely will not be able to
#do that at their apartments/flats.

In [ ]:
#Checking the distribution on Net Change in population according to continents
plot = sns.boxplot(data=data_chos2,
                   x='Continent',
                   y='Net Change')

#As we can see, there is a high net increase in population in Africa compared to other continents.

In [ ]:
#Checking Fert.Reta according to continents
plot = sns.boxplot(data=data_chos2,
                   x='Continent',
                   y='Fert. Rate')

#Same conclusions for fertility rate as in the previous graph. Africa's population will increase much more in the future
#compared to other countries.

### Analysis of cost of living

In [ ]:
data_cofl = pd.read_csv('./World Bank/countries-dataset-2020/Cost of living index by country 2020.csv')
data_cofl.head()

In [ ]:
#Checking for missing values
mis_val(data_cofl)

In [ ]:
name_cut(data_cofl, 'Country')
eu_flag(data_cofl, 'Country_Name_cut')
data_cofl_reg = pd.merge(data_cofl, data_reg, how= 'left', on = 'Country_Name_cut')
data_cofl.head(6)

In [ ]:
data_cofl = data_cofl[data_cofl['EU'] == 0]
data_cofl = data_cofl[['Country', 'Cost of Living Index','Rent Index', 'Cost of Living Plus Rent Index','Groceries Index','Restaurant Price Index','Local Purchasing Power Index','EU']]
data_cofl.head(6)

In [ ]:
data_cofl.describe()

In [ ]:
#Leaving only needed columns
data_cofl_reg = data_cofl_reg[['Country Name', 'Cost of Living Index','Rent Index', 'Cost of Living Plus Rent Index','Groceries Index','Restaurant Price Index','Local Purchasing Power Index','EU', 'Continent', 'Region']]
data_cofl_reg.head()

In [ ]:
#Filtering by top population countries
data_cofl2 = data_cofl_reg[data_cofl_reg['Country Name'].isin(list_cnt)]
data_cofl2_1 = data_cofl_reg[data_cofl_reg['Country Name'].isin(list_cnt2)]

#Reseting indexes
data_cofl2.reset_index(drop=True,inplace=True)
data_cofl2_1.reset_index(drop=True,inplace=True)

In [ ]:
#Filtering countries by mean cost of leaving to exclude those higher the mean value
dt_cofl_mean = data_cofl2_1[data_cofl2_1['Cost of Living Index']<data_cofl2_1['Cost of Living Index'].mean()].sort_values(by = 'Cost of Living Index', ascending=False)
dt_cofl_mean.reset_index(drop = True, inplace = True)
dt_cofl_mean

In [ ]:
#Plotting box for cost of living by continent
plot = sns.boxplot(data=data_cofl2_1,
                   x='Continent',
                   y='Cost of Living Index')

#Cost of living on average is higher in the North America compared to other continents. It means that customers will
#not be so tempted to buy sport clothing before everything else.

In [ ]:
#Looking closer to the data
dt_cofl_mean.describe().round(2)

In [ ]:
#Creating bar chart to visualize the cost of living for the chosen countries
axes2 = dt_cofl_mean.plot.bar(x = 'Country Name',y = 'Cost of Living Index',rot = 45)

In [ ]:
dt_cofl_mean[['Country Name','Cost of Living Index']].sort_values(by = 'Cost of Living Index',ascending=False)

### Analysis of GDP and GNI

In [ ]:
data_gdp0 = pd.read_csv('./World Bank/countries-of-the-world/countries of the world.csv')
data_gdp0['Country Name'] = data_gdp0['Country']
data_gdp0['GDP per capita, $'] = data_gdp0['GDP ($ per capita)']
data_gdp = data_gdp0[['Country Name','GDP per capita, $']].copy()

In [ ]:
#Uploading data of developed and developing countries to merge later
data_dev_ed = pd.read_csv('./World Bank/list_of_developed_countries.csv')
data_dev_ing = pd.read_csv('./World Bank/list_of_developed_or_developing_countries.csv')

In [ ]:
#Spliting to get correct Development Status
data_dev_ing[['Developed', 'Development Status']] = data_dev_ing['Development Status'].str.split('/', expand = True)

In [ ]:
#Getting only needed columns
data_dev_ing = data_dev_ing[['Country Name', 'GNI per capita', 'Human Development Index', 'Human Asset Index', 'Development Status', 'Population 2020']]

In [ ]:
data_gni = pd.merge(data_dev_ed, data_dev_ing, how = 'outer')
data_gni = data_gni[['Country Name', 'GNI per capita', 'Development Status']].copy()
data_gni[['GNI per capita', 'Currency']] = data_gni['GNI per capita'].str.split(' ', expand = True)

In [ ]:
data_gni['GNI per capita, $'] = data_gni['GNI per capita'].str.replace(',','')
data_gni['GNI per capita, $'] = pd.to_numeric(data_gni['GNI per capita, $'].str.replace('--',''))
data_gni = data_gni[['Country Name', 'GNI per capita, $','Development Status']]

In [ ]:
#Creating additional column of names for future merge
name_cut(data_gdp,'Country Name')
name_cut(data_gni, 'Country Name')

#Merging data
data_gdp_gni = pd.merge(data_gdp, data_gni, how= 'left', on = 'Country_Name_cut')
data_gdp_gni['Country Name'] = data_gdp_gni['Country Name_x']
data_gdp_gni = data_gdp_gni[['Country Name', 'Country_Name_cut', 'GDP per capita, $', 'GNI per capita, $', 'Development Status']]
data_gdp_gni.drop_duplicates(subset = 'Country_Name_cut', keep = 'first', inplace = True)

#Creating flag for EU
eu_flag(data_gdp_gni, 'Country_Name_cut')

data_gdp_gni = pd.merge(data_gdp_gni, data_reg, how = 'left', on = 'Country_Name_cut')

#Dealing with na's and renaming columns
data_gdp_gni = data_gdp_gni[['Country Name_x', 'GDP per capita, $', 'GNI per capita, $', 'Development Status', 'EU', 'Continent', 'Region']]
data_gdp_gni.drop(data_gdp_gni.tail(1).index, inplace = True)
data_gdp_gni.dropna(subset = ['GDP per capita, $', 'GNI per capita, $'], how = 'all', inplace = True)
data_gdp_gni.rename(columns = {'Country Name_x':'Country Name'}, inplace = True)
data_gdp_gni.head()

In [ ]:
#Exclude EU from the future analysis
gdp_gni_ne = data_gdp_gni[data_gdp_gni['EU'] == 0].copy()
gdp_gni_ne.reset_index(drop=True, inplace = True)
gdp_gni_ne.head()

In [ ]:
#Checking for na values
mis_val(gdp_gni_ne)

In [ ]:
#Filling numeric na with k-Neighbours
gdp_gni_ne_list = ['GDP per capita, $', 'GNI per capita, $']
gdp_gni_ne_nv = gdp_gni_ne[gdp_gni_ne_list]
impKNN = KNNImputer(n_neighbors=10)
newval = impKNN.fit_transform(gdp_gni_ne_nv)
gdp_gni_ne2 = pd.DataFrame(newval, columns=gdp_gni_ne_list, index = gdp_gni_ne_nv.index)

In [ ]:
#Working with data by merging, renaming columns and calculating GNI/GDP index for countries
gdp_gni_ne3 = gdp_gni_ne.merge(gdp_gni_ne2, left_index=True, right_index=True)
gdp_gni_ne3[['Country Name', 'GDP per capita, $', 'GNI per capita, $', 'Development Status', 'EU']] = gdp_gni_ne3[['Country Name', 'GDP per capita, $_y', 'GNI per capita, $_y', 'Development Status', 'EU']]
gdp_gni_ne3 = gdp_gni_ne3[['Country Name', 'GDP per capita, $', 'GNI per capita, $', 'Development Status', 'EU', 'Continent','Region']]
gdp_gni_ne3['GNI/GDP'] = round((gdp_gni_ne3['GNI per capita, $']/gdp_gni_ne3['GDP per capita, $']-1)*100,2)
gdp_gni_ne3.sort_values(by = ['GNI/GDP'], ascending=False, ignore_index=True).head()

In [ ]:
#Checking missing values again
mis_val(gdp_gni_ne3)

In [ ]:
#Finilaze data with only countries who have the biggest population
#name_cut(gdp_gni_ne3, 'Country Name')
gdp_gni_fin = gdp_gni_ne3[gdp_gni_ne3['Country Name'].astype(str).str.replace(' ','').str.capitalize().isin(list_cnt2)]
gdp_gni_fin.sort_values(by = 'GNI/GDP', ascending=False)

In [ ]:
mis_val(gdp_gni_fin)

In [ ]:
gdp_gni_fin.describe()

In [ ]:
#Preprocessing with data for future merge fpor 
data_chos_f['Country'] = data_chos_f['Country Name'].astype(str).str.replace(' ','').str.lower()
gdp_gni_fin['Country'] = gdp_gni_fin['Country Name'].astype(str).str.replace(' ','').str.lower()
dt_cofl_mean['Country'] = dt_cofl_mean['Country Name'].astype(str).str.replace(' ','').str.lower()



In [ ]:
#Finalizing the last table by merging everything
dt_final = pd.merge(data_chos_f, gdp_gni_fin, how= 'left', on = 'Country')
dt_final = pd.merge(dt_final, dt_cofl_mean)
dt_final = dt_final[['Country Name_x','Yearly Change %','World Share %', 'GNI/GDP','Cost of Living Index']]
dt_final.rename(columns = {'Country Name_x': 'Country Name'}, inplace=True)
dt_final.sort_values(by = ['GNI/GDP', 'Cost of Living Index','Yearly Change %', 'World Share %'], ascending=[False,False,False,False], ignore_index=True)

In [ ]:
dt_final.describe()

In [ ]:
axes3 = dt_final.plot.bar(x = 'Country Name',y = ['Yearly Change %','World Share %'],rot = 45)

In [ ]:
axes4 = dt_final.plot.bar(x = 'Country Name',y = 'Cost of Living Index',rot = 45)

In [ ]:
axes5 = dt_final.plot.bar(x = 'Country Name',y = 'GNI/GDP',rot = 45)

## Final decision

In [ ]:
#Choosing only countries with GNI/GDP bigger than 0, meaning that there are more international companies and it can be said 
#that it is safe to invest in these countries, as other market players also not afraid to invest in these economies
dt_final2 = dt_final[dt_final['GNI/GDP']>0]

#Leaving only countries with positive increase in population, bigger than 0.5%
dt_final2 = dt_final2[dt_final2['Yearly Change %']>0.5]

#Excluding countries with cost of living less than 35 (which is mean among these countries), as it can be too expansive
#for people there to buy company's products
dt_final2 = dt_final2[dt_final2['Cost of Living Index']<35]

dt_final2.sort_values(by = ['GNI/GDP','Cost of Living Index', 'Yearly Change %', 'World Share %'], ascending=[False,False,False,False], ignore_index=True)

In [ ]:
#Also if we want to decrease the number of possible countries, we can filter them by Population share in the world
dt_final2 = dt_final2[dt_final2['World Share %']>1.5]
dt_final2.sort_values(by = ['GNI/GDP', 'Yearly Change %', 'World Share %'], ascending=[False,False,False], ignore_index=True)

In [ ]:
#Final visualization of the data
ax = dt_final2.plot.bar(x = 'Country Name',y = ['Yearly Change %','World Share %'],rot = 45)

In [ ]:
ax2 = dt_final2.plot.bar(x = 'Country Name',y = ['Cost of Living Index'],rot = 45)

So we left with 2 countries (Nigeria is much more better option compared to Bangladesh) that are the best to open businesses there, as they have a high level of trust from other
companies (international investors, they have big enough population share with increasing rate, so we expect their
population to grow faster compared to other countries).

Also, China can be still added to this list as the only drawback according to this analysis is their growth rate of
population. Nevertheless, China has the biggest share of population, so its population growth rate has less influence and have a high trust of international investors with a quickly growing economy (based on GDP value).

As we can see, none of these countries are in the top sales countries. However, according to other aspects they are the best and can be count as easier markets to enter as there will not be a big competition with other sport companies and brands.